# Data Preparation
***

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
orig_df_clean = pd.read_csv('data/original_clean.csv',index_col='id')
res_clean  = pd.read_csv('data/residential_clean.csv',index_col='id')
parcel_clean  = pd.read_csv('data/parcel_clean.csv',index_col='id')
sales_clean  = pd.read_csv('data/sales_clean.csv',index_col='id')
ods_clean  = pd.read_csv('data/open_datasoft_clean.csv')
df = pd.read_csv('data/data_clean.csv')

### Before Merging
To prepare the datasets for merging, I:
- Made an `id` column out of the `Major` and `Minor` columns to create a primary key for merging.
- Extracted, cleaned, and filtered zip code information to be sure they are all in the same format and only include zip codes from King County.
- Standardized elements in categorical variables by renaming them to discrete numeric predictors. For example, renaming a "Yes/No" to 1's and 0's.
- Identifying duplicated `id` and in most cases dropping them as they represented only a small proportion of the data sets. In the `sales` dataset duplicates represented multiple transactions of the same property, so I only included the most recent transaction.
- Filtered data to only include properties labeled as Condominium or Residential in the parcel dataset.
- Filtered data to only include properties labeled condominiums, apartments, residential buildings, and mobile homes in the `sales` dataset.
- Extracted the year_sold from the `sales` data and filtered it to include sales since 2021 to account for only recent data when making the model.
- Standardized column names for merging.

In [2]:
orig_df_clean.head(3)

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,greenbelt,nuisance,view,condition,grade,sqft_basement,yr_built,yr_renovated,zipcode
id,,,,,,,,,,,,,,,,
7399300360,675000.0,4,1.0,1180,7140,1.0,0,0,0,0,4,7,0,1969,0,98055
8910500230,920000.0,5,2.5,2770,6703,1.0,0,0,1,3,3,7,1570,1950,0,98133
1180000275,311000.0,6,2.0,2880,6156,1.0,0,0,0,3,3,7,1580,1956,0,98178


In [3]:
res_clean.head(3)

,bathrooms,bedrooms,condition,floors,grade,sqft_basement,sqft_living,yr_built,yr_renovated,zipcode
id,,,,,,,,,,
126039624,2,3,3,1.0,8,720,1810,1982,0,98133.0
126039625,2,3,3,1.5,7,2320,4340,1994,0,98177.0
126039628,1,4,3,1.0,7,660,1800,1982,0,98133.0


In [4]:
parcel_clean.head(3)

,greenbelt,nuisance,sqft_lot,view,waterfront
id,,,,,
7147600085,0,0,16693,0,0.0
7399200210,0,0,8686,0,0.0
5101408598,0,0,6434,0,0.0


In [5]:
sales_clean.head(3)

,date,price
id,,
1000035,2021-08-18,0
1000055,2021-10-20,248448
1000061,2021-03-24,475000


In [6]:
ods_clean.head(3)

,zipcode,population,density
0,98029,29250.0,1261.5
1,98070,10291.0,107.6
2,98074,28775.0,1041.4


### After Merging
After combining the data into a single dataframe, I:
- Cleaned columns to only include reasonable data. For example, including dropping any rows where `price` was equal to or less than 0. 
- Created a new column, `yr_last_construction`, which took the year from `yr_renovated` if renovated and `yr_built` if not. 
- Removed any rows with a `zipcode` count of less than or equal to 20 as they would be too small to count as a sample. 
- Removed any outliers. This made sense as extreme values could bias the statistics and are unlikely.
- Created new columns with all numerical variables normalized to be used for modeling.

In [7]:
df.head()

,id,price,bedrooms,bedrooms_norm,bathrooms,bathrooms_norm,sqft_living,sqft_living_norm,sqft_lot,sqft_lot_norm,...,view,view_norm,waterfront,greenbelt,nuisance,condition,yr_built,yr_renovated,zipcode,yr_last_construction
0,1000055,248448,3,-0.461562,1,-0.831197,1790,-0.317835,39808,4.503585,...,0,-0.34215,0,0,0,4,1958,0,98002,1958
1,1000061,475000,3,-0.461562,1,-0.831197,1870,-0.228085,9500,0.094375,...,0,-0.34215,0,0,0,5,1958,0,98002,1958
2,1000070,610000,3,-0.461562,1,-0.831197,1790,-0.317835,19088,1.489238,...,0,-0.34215,0,0,0,4,1958,0,98002,1958
3,1000094,609000,4,0.645705,1,-0.831197,2232,0.178032,12866,0.584061,...,0,-0.34215,0,0,0,4,1934,0,98002,1934
4,1000095,300000,2,-1.568829,1,-0.831197,960,-1.248988,7380,-0.214043,...,0,-0.34215,0,0,0,4,1914,0,98002,1914


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48682 entries, 0 to 48681
Data columns (total 30 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    48682 non-null  int64  
 1   price                 48682 non-null  int64  
 2   bedrooms              48682 non-null  int64  
 3   bedrooms_norm         48682 non-null  float64
 4   bathrooms             48682 non-null  int64  
 5   bathrooms_norm        48682 non-null  float64
 6   sqft_living           48682 non-null  int64  
 7   sqft_living_norm      48682 non-null  float64
 8   sqft_lot              48682 non-null  int64  
 9   sqft_lot_norm         48682 non-null  float64
 10  sqft_basement         48682 non-null  int64  
 11  sqft_basement_norm    48682 non-null  float64
 12  floors                48682 non-null  int64  
 13  floors_norm           48682 non-null  float64
 14  grade                 48682 non-null  int64  
 15  grade_norm         